In [2]:
import sys
sys.path.append("/home/sidhu/Projects/tf-transformers/src/")

In [3]:
from tf_transformers.data import TFWriter, TFReader, TFProcessor

In [4]:
import tensorflow as tf
from tf_transformers.data.processors.mlm import (
    dynamic_causal_lm_from_features,
    dynamic_masking_from_features,
    dynamic_prefix_lm_from_features,
)

In [8]:
from tf_transformers.data.utils import auto_batch
from tf_transformers.utils import tf_utils

In [5]:
# Create dummy tfrecord

def parse_train():
    for i in range(1000):
        random_length = tf.random.uniform(minval=10, maxval=128, shape=(1,), dtype=tf.int32)[0]
        vector = tf.random.uniform(minval=0, maxval=10000,shape=(random_length,), dtype=tf.int32)
        vector = vector.numpy().tolist()
        yield {"input_ids": vector}
    
    
schema = {
    "input_ids": ("var_len", "int"),
}

tfrecord_train_dir = 'tfrecord_dummy'
tfrecord_filename = 'dummy'
tfwriter = TFWriter(schema=schema, 
                    file_name=tfrecord_filename, 
                    model_dir=tfrecord_train_dir,
                    tag='train',
                    n_files=1,
                    overwrite=True
                    )
# Process
tfwriter.process(parse_fn=parse_train())


INFO:absl:Total individual observations/examples written is 1000 in 2.317816972732544 seconds
INFO:absl:All writer objects closed


In [6]:
# Read tfrecord
import glob
def get_tfdataset_from_tfrecords(tfrecord_path_list):
    """Get tf dataset from tfrecords"""
    all_files = []
    for tfrecord_path in tfrecord_path_list:
        all_files.extend(glob.glob("{}/*.tfrecord".format(tfrecord_path)))
    schema    = json.load(open("{}/schema.json".format(tfrecord_path)))
    tf_reader = TFReader(schema=schema, 
                        tfrecord_files=all_files)
    train_dataset = tf_reader.read_record(
                                      )
    return train_dataset

dataset = get_tfdataset_from_tfrecords([tfrecord_train_dir])

In [7]:
for item in dataset:
    print(item)
    break

{'input_ids': <tf.Tensor: shape=(93,), dtype=int32, numpy=
array([5952, 3643, 7967, 6195, 3369, 2210, 9291, 2911, 5740, 3711, 6952,
       1476, 2407, 5838, 6559, 2477, 4107, 2194,  120, 5211, 5620,  667,
       1644, 9636, 1329, 3799, 5731, 7563, 7050, 1720, 6437, 6717, 1020,
       2492, 9300,  948, 6357, 2033, 8648, 2489, 6808, 2274, 6029, 4729,
       5366, 4358, 2885, 9429, 4410, 7672, 8602, 5081, 2272, 1845, 6607,
       6031, 5429, 8123, 5859, 1106, 7327, 3914, 5751, 1728, 2399,   42,
       4557, 6764, 1194, 4671, 1107, 2211, 5078, 6014, 8458,  408, 5195,
       8810, 7499, 8514, 6822, 6580, 5549, 4560, 8958, 3485, 9272, 6463,
        769, 5415, 4016, 4280,   57], dtype=int32)>}


In [98]:
def get_dataset(
    tfrecord_path_list,
    max_seq_len,
    max_predictions_per_batch,
    vocab_size,
    cls_token_id,
    sep_token_id,
    unk_token_id,
    pad_token_id,
    mask_token_id,
    batch_size,
    min_sen_len,
):
    """Get dataset after mlm from TFRecords"""

    def filter_by_length(x, min_sen_len):
        """Filter by minimum sentence length (subwords)"""
        return tf.squeeze(tf.greater_equal(tf.shape(x['input_ids']), tf.constant(min_sen_len)), axis=0)

    def filter_by_batch(x, y, batch_size):
        """Filter by batch size"""
        x_batch = tf.shape(x['input_ids'])[0]
        return tf.equal(x_batch, tf.constant(batch_size))
    
    def prepare_3d_input_mask_mlm(input_mask):
        """Prepare 3D mask from 2D"""
        batch_size = tf.shape(input_mask)[0]
        seq_length = tf.shape(input_mask)[1]

        to_mask = tf.cast(tf.reshape(input_mask, [batch_size, 1, seq_length]), dtype=input_mask.dtype)
        broadcast_ones = tf.ones(shape=[batch_size, seq_length, 1], dtype=input_mask.dtype)

        mask = broadcast_ones * to_mask

        return tf.cast(mask, tf.float32)
    
    # Dynamic MLM
    dynamic_mlm_fn = dynamic_masking_from_features(
        max_seq_len,
        max_predictions_per_batch,
        vocab_size,
        cls_token_id,
        sep_token_id,
        unk_token_id,
        pad_token_id,
        mask_token_id,
    )
    

    # Dynamic Prefix LM
    dynamic_prefix_lm = dynamic_prefix_lm_from_features(max_seq_len, 
                                    cls_token_id, sep_token_id)
    
    # Dynamic Causal LM
    dynamic_causal_lm = dynamic_causal_lm_from_features(max_seq_len, 
                                    cls_token_id, sep_token_id)
    
    train_dataset = get_tfdataset_from_tfrecords(tfrecord_path_list)

    if min_sen_len and min_sen_len > 0:
        train_dataset = train_dataset.filter(lambda x: filter_by_length(x, min_sen_len))
    
    # prob check has to be inside map
    # otherwise things become deterministic
    def get_dataset_based_on_prob(item):
        """Map function"""
        
        def add_mark(x, mode, prob):
            """Check are we getting all if conditions with equal probability"""
            x['mode'] = [mode]
            x['prob'] = [prob]
            return x
        
        def map_mlm(x):
            """MLM"""
            x['input_ids'] = tf.RaggedTensor.from_tensor(tf.expand_dims(x['input_ids'], axis=0))
            x_copy , y_copy = dynamic_mlm_fn(x)
            x = {}
            for name, v_tensor in x_copy.items():
                x[name] = tf.squeeze(v_tensor, axis=0)
            y = {}
            for name, v_tensor in y_copy.items():
                y[name] = tf.squeeze(v_tensor, axis=0)
            x['3d_mask']   = tf.squeeze(prepare_3d_input_mask_mlm(x_copy['input_mask']), axis=0)
            
            for name, v_tensor in y.items():
                x[name] = v_tensor
            return x
        
        def map_pcmlm(x):
            """Prefix Causal LM"""
            x, y = dynamic_prefix_lm(x)
            for name, v_tensor in y.items():
                x[name] = v_tensor
            return x
        
        def map_cmlm(x):
            """Causal LM"""
            x, y = dynamic_causal_lm(x)
            for name, v_tensor in y.items():
                x[name] = v_tensor
            return x
    

        prob = tf.random.uniform(shape=())
        # Keep a copy like this importatnt
        # otherwise transformation in first if cond might affect other
        input_ids = item['input_ids']
        
        # Do MLM
        if prob <= 0.33:
            x = map_mlm(item)
            x['masked_lm_positions'] = tf.cast(x['masked_lm_positions'], dtype=tf.int32)
            x['masked_lm_weights']   = tf.cast(x['masked_lm_weights'], dtype=tf.int32)
            x['input_mask'] = x['3d_mask']
            del x['3d_mask']
            x = add_mark(x, "mlm", prob)
            
        # Prefix CLM
        elif prob < 0.66:
            x = map_pcmlm({"input_ids": input_ids})
            x['input_mask'] = x['3d_mask']
            del x['3d_mask']            
            x = add_mark(x, "prefix", prob)
            
        else:
            x = map_cmlm({"input_ids": input_ids})
            x['input_mask'] = x['3d_mask']
            del x['3d_mask']
            x = add_mark(x, "causal", prob)
        return x
    
    
    train_dataset = train_dataset.map(get_dataset_based_on_prob, num_parallel_calls=tf.data.AUTOTUNE)
    train_dataset = auto_batch(train_dataset, 
                              batch_size,
                                x_keys=['input_ids', 'input_type_ids', 'input_mask', 'masked_lm_positions'],
                                y_keys=['masked_lm_labels', 'masked_lm_weights', 'mode', 'prob'],
                              shuffle=True
                              )
    train_dataset = train_dataset.filter(lambda x, y: filter_by_batch(x, y, batch_size))
    train_dataset = train_dataset.shuffle(100)
    train_dataset = train_dataset.prefetch(100)

    return train_dataset

In [99]:
max_seq_len = 128
max_predictions_per_batch = 20
vocab_size = 30200
cls_token_id = 2
sep_token_id = 3
unk_token_id = 1
pad_token_id = 0
mask_token_id = 5
batch_size = 5
min_sen_len = None
train_dataset = get_dataset(
    [tfrecord_train_dir],
    max_seq_len,
    max_predictions_per_batch,
    vocab_size,
    cls_token_id,
    sep_token_id,
    unk_token_id,
    pad_token_id,
    mask_token_id,
    batch_size,
    min_sen_len,
)

Padded shapes {'input_ids': [None], 'input_type_ids': [None], 'input_mask': [None, None], 'masked_lm_positions': [None], 'masked_lm_labels': [None], 'masked_lm_weights': [None], 'mode': [None], 'prob': [None]}


In [100]:
all_modes = []
all_probs = []
for (batch_inputs, batch_labels) in train_dataset:
    all_modes.extend(batch_labels['mode'].numpy())
    all_probs.extend(batch_labels['prob'].numpy())
    break

In [ ]:
from collections import Counter

Counter(all_modes)

In [101]:
all_modes

[array([b'mlm'], dtype=object),
 array([b'causal'], dtype=object),
 array([b'prefix'], dtype=object),
 array([b'mlm'], dtype=object),
 array([b'prefix'], dtype=object)]

In [102]:
batch_inputs

{'input_ids': <tf.Tensor: shape=(5, 128), dtype=int32, numpy=
 array([[    2,  8725,  8350,  9012,  9594,  4473,  2253,  7767,  6496,
          3794,  3973,  6197,  3968,  1056,  4873,    81,  4731,  8508,
             5,  6532,  4748,  1706,  6403,  8314,  7474,     5,  9873,
          9950,  7376,     5,  1927,  2806,  7745,  1511,   982, 15391,
          7896,     5,  7769,     5,  8273,   663,  7205,  6093,  4153,
          6862,  3223,  5360, 20256,  5962,  9436,  7603, 17984,     5,
          8310,  6297,  2921,  2230,  7995,     5,  8583,   783,  6228,
             5,     5,  1044,  8177,  7197,  2025,  2296,  8295,  8599,
             5,  4027,  2449,  7628,     3,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
  

In [104]:
tf.reduce_sum(batch_inputs['input_mask'][1], axis=0)

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([120., 119., 118., 117., 116., 115., 114., 113., 112., 111., 110.,
       109., 108., 107., 106., 105., 104., 103., 102., 101., 100.,  99.,
        98.,  97.,  96.,  95.,  94.,  93.,  92.,  91.,  90.,  89.,  88.,
        87.,  86.,  85.,  84.,  83.,  82.,  81.,  80.,  79.,  78.,  77.,
        76.,  75.,  74.,  73.,  72.,  71.,  70.,  69.,  68.,  67.,  66.,
        65.,  64.,  63.,  62.,  61.,  60.,  59.,  58.,  57.,  56.,  55.,
        54.,  53.,  52.,  51.,  50.,  49.,  48.,  47.,  46.,  45.,  44.,
        43.,  42.,  41.,  40.,  39.,  38.,  37.,  36.,  35.,  34.,  33.,
        32.,  31.,  30.,  29.,  28.,  27.,  26.,  25.,  24.,  23.,  22.,
        21.,  20.,  19.,  18.,  17.,  16.,  15.,  14.,  13.,  12.,  11.,
        10.,   9.,   8.,   7.,   6.,   5.,   4.,   3.,   2.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.], dtype=float32)>

In [448]:
from tf_transformers.models import BertEncoder
from tf_transformers.core import LegacyModel

In [449]:
class MixEncoder(BertEncoder):
    
    def __init__(self, config, **kwargs):
        print(kwargs)
        super(MixEncoder, self).__init__(config, **kwargs)
        
    def get_model(self, initialize_only=False):
        """Convert tf.keras.Layer to a tf.keras.Model/LegacyModel.
        Args:
            self: model (tf.keras.Layer) instance
        """

        input_ids = tf.keras.layers.Input(
            shape=(self._sequence_length,),
            batch_size=self._batch_size,
            dtype=tf.int32,
            name="input_ids",
        )
        input_mask = tf.keras.layers.Input(
            shape=(self._sequence_length,self._sequence_length),
            batch_size=self._batch_size,
            dtype=tf.float32,
            name="input_mask",
        )
        input_type_ids = tf.keras.layers.Input(
            shape=(self._sequence_length,),
            batch_size=self._batch_size,
            dtype=tf.int32,
            name="input_type_ids",
        )
        masked_lm_positions = tf.keras.layers.Input(
            shape=(None,),
            batch_size=self._batch_size,
            dtype=tf.int32,
            name="masked_lm_positions",
        )
        inputs = {}
        inputs["input_ids"] = input_ids  # Default
        # if mask_mode != 'causal', user has to provde mask
        if self._mask_mode != "causal":
            inputs["input_mask"] = input_mask
        # If type mebddings required
        if self._type_embeddings_layer:
            inputs["input_type_ids"] = input_type_ids
        # if masked_lm_positions
        if self._use_masked_lm_positions:
            inputs["masked_lm_positions"] = masked_lm_positions


        layer_outputs = self(inputs)
        if initialize_only:
            return inputs, layer_outputs

        # Adding model_config is a hack
        model = LegacyModel(inputs=inputs, outputs=layer_outputs, name=self._model_name)
        model.model_config = self._config_dict
        return model

    def call_encoder(self, inputs):
        """Forward pass of an Encoder

        Args:
            inputs ([dict of tf.Tensor]): This is the input to the model.

            'input_ids'         --> tf.int32 (b x s)
            'input_mask'        --> tf.int32 (b x s) # optional
            'input_type_ids'    --> tf.int32 (b x s) # optional

        Returns:
            [dict of tf.Tensor]: Output from the model

            'cls_output'        --> tf.float32 (b x s) # optional
            'token_embeddings'  --> tf.float32 (b x s x h)
            'all_layer_token_embeddings' --> tf.float32 (List of (b x s x h)
                                              from all layers)
            'all_layer_cls_output'       --> tf.float32 (List of (b x s)
                                              from all layers)
        """

        # 1. Collect Word Embeddings
        input_ids = inputs["input_ids"]
        sequence_length = tf.shape(input_ids)[1]
        embeddings = self._embedding_layer(input_ids)
        # Add word_embeddings + position_embeddings + type_embeddings
        if self._type_embeddings_layer:
            input_type_ids = inputs["input_type_ids"]
            type_embeddings = self._type_embeddings_layer(input_type_ids)
            embeddings = embeddings + type_embeddings
        if self._positional_embedding_layer:
            positional_embeddings = self._positional_embedding_layer(tf.range(sequence_length))
            embeddings = embeddings + positional_embeddings

        # 2. Norm + dropout
        embeddings = self._embedding_norm(embeddings)
        embeddings = self._embedding_dropout(embeddings, training=self._use_dropout)

        # 3. Attention  Mask
        attention_mask = inputs['input_mask']

        # 4. Transformer Outputs
        encoder_outputs = []
        for i in range(self._config_dict["num_hidden_layers"]):
            layer = self._transformer_layers[i]
            embeddings, _, _ = layer([embeddings, attention_mask])
            encoder_outputs.append(embeddings)

        # First word of last layer outputs [CLS]
        cls_token_tensor = tf.keras.layers.Lambda(lambda x: tf.squeeze(x[:, 0:1, :], axis=1))(encoder_outputs[-1])
        # batch_size x embedding_size
        cls_output = self._pooler_layer(cls_token_tensor)
        # batch_size x sequence_length x embedding_size
        token_embeddings = encoder_outputs[-1]

        # check for masked lm positions
        # only for encoder forward pass. This is for MaskedLM training
        if "masked_lm_positions" in inputs:
            masked_lm_positions = inputs["masked_lm_positions"]
        else:
            masked_lm_positions = None

        # MaskedLM layer only project it and normalize (b x s x h)
        token_embeddings_mlm = self._masked_lm_layer(token_embeddings, masked_lm_positions)
        token_logits = tf.matmul(
            token_embeddings_mlm, tf.cast(self.get_embedding_table(), dtype=tf_utils.get_dtype()), transpose_b=True
        )
        # token_logits         =  tf.nn.bias_add(token_logits, self._masked_lm_bias)
        token_logits = self._masked_lm_bias(token_logits)
        last_token_logits = tf.keras.layers.Lambda(lambda x: x[:, -1, :])(token_logits)

        result = {
            "cls_output": cls_output,
            "token_embeddings": token_embeddings,
            "token_logits": token_logits,
            "last_token_logits": last_token_logits,
        }

        if self._return_all_layer_outputs:
            all_cls_output = []
            all_token_logits = []
            for per_layer_token_embeddings in encoder_outputs:
                per_cls_token_tensor = tf.keras.layers.Lambda(lambda x: tf.squeeze(x[:, 0:1, :], axis=1))(
                    per_layer_token_embeddings
                )
                all_cls_output.append(self._pooler_layer(per_cls_token_tensor))

                # token logits per layer
                layer_token_embeddings_mlm = self._masked_lm_layer(per_layer_token_embeddings, masked_lm_positions)
                layer_token_logits = tf.matmul(
                    layer_token_embeddings_mlm,
                    tf.cast(self.get_embedding_table(), dtype=tf_utils.get_dtype()),
                    transpose_b=True,
                )
                layer_token_logits = self._masked_lm_bias(layer_token_logits)
                all_token_logits.append(layer_token_logits)

            result["all_layer_token_embeddings"] = encoder_outputs
            result["all_layer_cls_output"] = all_cls_output
            result["all_layer_token_logits"] = all_token_logits

        return result


In [450]:
config = {
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "intermediate_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "embedding_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "max_position_embeddings": 512,
    "num_attention_heads": 12,
    "attention_head_size": 64,
    "num_hidden_layers": 12,
    "type_vocab_size": 2,
    "vocab_size": 30000,
    "layer_norm_epsilon": 1e-12
}

tf.keras.backend.clear_session()
model = MixEncoder(config,
                         is_training=True,
                         use_dropout=True,
                         use_masked_lm_positions=True,
                         return_all_layer_outputs=True)
model = model.get_model()

{'is_training': True, 'use_dropout': True, 'use_masked_lm_positions': True, 'return_all_layer_outputs': True}


In [451]:
model.input

{'input_ids': <KerasTensor: shape=(None, None) dtype=int32 (created by layer 'input_ids')>,
 'input_mask': <KerasTensor: shape=(None, None, None) dtype=float32 (created by layer 'input_mask')>,
 'input_type_ids': <KerasTensor: shape=(None, None) dtype=int32 (created by layer 'input_type_ids')>,
 'masked_lm_positions': <KerasTensor: shape=(None, None) dtype=int32 (created by layer 'masked_lm_positions')>}

In [452]:
model.output

{'cls_output': <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_transformers/bert')>,
 'token_embeddings': <KerasTensor: shape=(None, None, 768) dtype=float32 (created by layer 'tf_transformers/bert')>,
 'token_logits': <KerasTensor: shape=(None, None, 30000) dtype=float32 (created by layer 'tf_transformers/bert')>,
 'last_token_logits': <KerasTensor: shape=(None, 30000) dtype=float32 (created by layer 'tf_transformers/bert')>,
 'all_layer_token_embeddings': [<KerasTensor: shape=(None, None, 768) dtype=float32 (created by layer 'tf_transformers/bert')>,
  <KerasTensor: shape=(None, None, 768) dtype=float32 (created by layer 'tf_transformers/bert')>,
  <KerasTensor: shape=(None, None, 768) dtype=float32 (created by layer 'tf_transformers/bert')>,
  <KerasTensor: shape=(None, None, 768) dtype=float32 (created by layer 'tf_transformers/bert')>,
  <KerasTensor: shape=(None, None, 768) dtype=float32 (created by layer 'tf_transformers/bert')>,
  <KerasTensor: shape=(None, 

In [460]:
outputs = model(batch_inputs)

In [461]:
outputs.keys()

dict_keys(['cls_output', 'token_embeddings', 'token_logits', 'last_token_logits', 'all_layer_token_embeddings', 'all_layer_cls_output', 'all_layer_token_logits'])